<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=8dcbc918501717c19caebc8ad7e7de08303b16cc46e1261e5846edfd3a3f70fb
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-13 10:12:27
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: -1.06 L (-0.75%)
Current PnL: -20.83 L (-13.88%)
CY Booked + Current PnL: -9.39 L (-6.26%)
-------------------
Total profit:  1.63 L
Total loss:  -22.46 L
-------------------
Total Booked + Current PnL: 17.59 L (14.28%)
Total Booked PnL: 38.42 L (31.2%)
Curr Year Booked PnL: 11.44 L (8.13%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.34 L (62.76%)
Deployed:  1.23 C
Current:  1.41 C
CAGR/XIRR %: 8.0%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,65.0,H-MC,14.55,119239.0,18735.0,10374.0,0.30,18.64,8.70,28.96,79.0,1.81,0.85,58.94,MH,ATH,METALS
19,CIPLA,1662.74,-20.12,56.0,X-LC,3.68,213131.0,8631.0,14663.0,-0.39,4.22,6.88,11.39,10.0,0.59,1.52,13.55,X40N,ATH,PHARMA
77,TTKPRESTIG,770.00,94.45,43.0,M-SC,1.24,83735.0,-17042.0,17132.0,0.62,-16.91,20.46,0.09,245.0,-0.99,0.60,10.01,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,10.40,50.0,X-MC,3.33,190500.0,25830.0,17507.0,-2.26,15.69,9.19,26.31,89.0,1.48,1.36,24.92,X40N,NTT,AC
84,VOLTAS,1530.00,-0.52,54.0,X-MC,1.10,208785.0,17043.0,20711.0,-1.79,8.89,9.92,19.69,99.0,0.82,1.49,16.29,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIS,528.00,1980.83,35.0,H-SC,3.85,83984.0,-27048.0,50659.0,1.26,-24.36,60.32,21.26,156.0,-0.53,0.60,13.47,OX40N,NTT,MISC
78,UJJIVANSFB,60.00,108.87,64.0,H-SC,13.79,131706.0,-10773.0,30292.0,1.18,-7.56,23.00,13.70,163.0,-0.36,0.94,55.45,OX40N,NTT,BANKS
77,TTKPRESTIG,770.00,94.45,43.0,M-SC,1.24,83735.0,-17042.0,17132.0,0.62,-16.91,20.46,0.09,245.0,-0.99,0.60,10.01,OX40N,NTT,DURABLES
37,INDIAMART,4810.62,-54.27,44.0,H-SC,8.52,127486.0,4150.0,127473.0,0.59,3.36,99.99,106.72,119.0,0.03,0.91,27.53,AR,ATH,MISC
33,HONAUT,58357.33,-22.02,51.0,X-SC,2.15,109890.0,-18048.0,65187.0,0.42,-14.11,59.32,36.84,143.0,-0.28,0.78,12.50,X40N,ATH,ELECTRICAL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.0,-43.66,50.0,L-SC,29.06,81984.0,-31565.0,71613.0,-3.20,-27.80,87.35,35.27,268.0,-0.44,0.58,100.66,XR,NTT,HOTELS
55,RAJOOENG,143.1,-40.67,46.0,H-SC,16.77,89800.0,-12700.0,53296.0,-3.08,-12.39,59.35,39.61,114.0,-0.24,0.64,9.67,AR,ATH,MISC
82,VALIANTORG,838.0,-288.40,28.0,H-SC,2.34,104720.0,-58885.0,180202.0,-2.69,-35.99,172.08,74.15,139.0,-0.33,0.75,34.23,XR,NTT,CHEMICALS
14,BLUESTARCO,2080.0,10.40,50.0,X-MC,3.33,190500.0,25830.0,17507.0,-2.26,15.69,9.19,26.31,89.0,1.48,1.36,24.92,X40N,NTT,AC
65,SIEMENS,4671.5,-4.11,40.0,H-LC,1.29,153940.0,-32155.0,79633.0,-2.18,-17.28,51.73,25.51,15.0,-0.40,1.10,13.44,AR,ATH,ELECTRICAL


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.0,42.74,48.0,M-MC,12.60,225909.0,947.0,166698.0,-0.74,0.42,73.79,74.53,192.0,0.01,1.61,32.81,XY24,BTT,STEEL
86,WIPRO,420.0,-13.34,49.0,M-LC,5.82,152253.0,1308.0,108145.0,-1.26,0.87,71.03,72.51,53.0,0.01,1.09,6.94,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,108.87,64.0,H-SC,13.79,131706.0,-10773.0,30292.0,1.18,-7.56,23.00,13.70,163.0,-0.36,0.94,55.45,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,94.45,43.0,M-SC,1.24,83735.0,-17042.0,17132.0,0.62,-16.91,20.46,0.09,245.0,-0.99,0.60,10.01,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,49.0,H-SC,1.98,221139.0,-48528.0,84851.0,-0.72,-18.00,38.37,13.47,138.0,-0.57,1.58,12.44,XY24,NTT,PAINTS
18,CERA,9475.0,-25.30,30.0,H-SC,1.94,136114.0,-39789.0,81805.0,-0.82,-22.62,60.10,23.89,149.0,-0.49,0.97,17.55,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-25.39,41.0,H-MC,6.85,100985.0,-30850.0,73073.0,-1.71,-23.40,72.36,32.03,98.0,-0.42,0.72,13.11,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,42.74,48.0,M-MC,12.60,225909.0,947.0,166698.0,-0.74,0.42,73.79,74.53,192.0,0.01,1.61,32.81,XY24,BTT,STEEL
32,HINDZINC,730.22,32.99,65.0,M-LC,8.24,217174.0,12098.0,100465.0,0.19,5.90,46.26,54.89,52.0,0.12,1.55,31.42,X5K,ATH,METALS
79,UNIONBANK,163.00,-13.18,56.0,M-LC,9.35,157918.0,17078.0,27415.0,-0.27,12.13,17.36,31.59,66.0,0.62,1.13,41.83,XY24,NTT,BANKS
86,WIPRO,420.00,-13.34,49.0,M-LC,5.82,152253.0,1308.0,108145.0,-1.26,0.87,71.03,72.51,53.0,0.01,1.09,6.94,XR,NTT,IT
11,BANKINDIA,190.00,-25.47,65.0,H-MC,12.31,193665.0,13857.0,98556.0,-0.67,7.71,50.89,62.52,88.0,0.14,1.38,44.42,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,65.0,H-MC,14.55,119239.0,18735.0,10374.0,0.30,18.64,8.70,28.96,79.0,1.81,0.85,58.94,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,65.0,H-MC,12.31,193665.0,13857.0,98556.0,-0.67,7.71,50.89,62.52,88.0,0.14,1.38,44.42,XR,NTT,BANKS
32,HINDZINC,730.22,32.99,65.0,M-LC,8.24,217174.0,12098.0,100465.0,0.19,5.90,46.26,54.89,52.0,0.12,1.55,31.42,X5K,ATH,METALS
37,INDIAMART,4810.62,-54.27,44.0,H-SC,8.52,127486.0,4150.0,127473.0,0.59,3.36,99.99,106.72,119.0,0.03,0.91,27.53,AR,ATH,MISC
86,WIPRO,420.00,-13.34,49.0,M-LC,5.82,152253.0,1308.0,108145.0,-1.26,0.87,71.03,72.51,53.0,0.01,1.09,6.94,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,DABUR,735.00,-13.02,34.0,X-MC,3.56,235613.0,-12455.0,120869.0,-0.80,-5.02,51.30,43.70,102.0,-0.10,1.68,8.70,XY24,BTT,FMCG
29,HAVELLS,2069.16,-7.03,34.0,X-MC,0.88,230537.0,-17194.0,92261.0,-1.21,-6.94,40.02,30.30,92.0,-0.19,1.64,5.43,X40,ATH,ELECTRICAL
9,BAJAJHFL,181.50,-15.11,35.0,X-MC,5.14,178826.0,-21680.0,118830.0,-0.87,-10.81,66.45,48.45,90.0,-0.18,1.28,3.12,X40N,ATH,FINANCE
5,ASIANPAINT,3465.66,-18.16,36.0,X-LC,5.47,206642.0,-45126.0,98341.0,0.34,-17.92,47.59,21.13,27.0,-0.46,1.47,11.15,X40,ATH,PAINTS
53,QUESS,986.00,-46.01,36.0,X-SC,37.70,53567.0,-11439.0,162367.0,-0.27,-17.60,303.11,232.18,198.0,-0.07,0.38,2.77,XY24,NTT,MISC


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,HAVELLS,2069.16,-7.03,34.0,X-MC,0.88,230537.0,-17194.0,92261.0,-1.21,-6.94,40.02,30.30,92.0,-0.19,1.64,5.43,X40,ATH,ELECTRICAL
84,VOLTAS,1530.00,-0.52,54.0,X-MC,1.10,208785.0,17043.0,20711.0,-1.79,8.89,9.92,19.69,99.0,0.82,1.49,16.29,XY25,NTT,AC
73,TATAMOTORS,1065.00,-22.82,41.0,X-LC,1.22,243936.0,-32587.0,142654.0,-1.02,-11.78,58.48,39.81,54.0,-0.23,1.74,16.91,XY24,NTT,AUTO
65,SIEMENS,4671.50,-4.11,40.0,H-LC,1.29,153940.0,-32155.0,79633.0,-2.18,-17.28,51.73,25.51,15.0,-0.40,1.10,13.44,AR,ATH,ELECTRICAL
34,ICICIGI,2260.25,-20.34,41.0,X-MC,1.65,137152.0,1019.0,30105.0,-0.50,0.75,21.95,22.86,91.0,0.03,0.98,13.46,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-21.40,39.0,X-LC,10.46,283081.0,-32761.0,145419.0,0.01,-10.37,51.37,35.67,5.0,-0.23,2.02,2.04,X40N,ATH,FMCG
53,QUESS,986.00,-46.01,36.0,X-SC,37.70,53567.0,-11439.0,162367.0,-0.27,-17.60,303.11,232.18,198.0,-0.07,0.38,2.77,XY24,NTT,MISC
22,COLPAL,3767.14,-4.98,38.0,X-MC,7.45,221760.0,-41605.0,154944.0,-0.46,-15.80,69.87,43.04,84.0,-0.27,1.58,2.95,XY25,ATH,FMCG
9,BAJAJHFL,181.50,-15.11,35.0,X-MC,5.14,178826.0,-21680.0,118830.0,-0.87,-10.81,66.45,48.45,90.0,-0.18,1.28,3.12,X40N,ATH,FINANCE
42,ITC,452.00,-39.19,43.0,X-LC,2.55,195796.0,-4342.0,25238.0,-0.60,-2.17,12.89,10.44,4.0,-0.17,1.40,3.27,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-46.01,36.0,X-SC,37.70,53567.0,-11439.0,162367.0,-0.27,-17.60,303.11,232.18,198.0,-0.07,0.38,2.77,XY24,NTT,MISC
56,RELAXO,1176.00,-40.36,37.0,X-SC,2.86,81329.0,-63831.0,139756.0,-0.46,-43.97,171.84,52.31,136.0,-0.46,0.58,8.86,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-35.29,42.0,X-SC,4.38,96695.0,-31975.0,77269.0,-0.34,-24.85,79.91,35.20,219.0,-0.41,0.69,10.98,X40,NTT,FOOTWEAR
17,CAMS,5211.76,-4.72,52.0,X-SC,1.20,108307.0,6303.0,37626.0,0.19,6.18,34.74,43.06,122.0,0.17,0.77,24.35,X40N,ATH,MISC
33,HONAUT,58357.33,-22.02,51.0,X-SC,2.15,109890.0,-18048.0,65187.0,0.42,-14.11,59.32,36.84,143.0,-0.28,0.78,12.50,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-28.69,49.0,X-LC,12.17,283720.0,-62236.0,130483.0,-0.33,-17.99,45.99,19.73,1.0,-0.48,2.02,4.50,X40,ATH,IT
40,INFY,2275.00,-18.16,53.0,X-LC,7.63,318669.0,5781.0,165899.0,-1.24,1.85,52.06,54.87,3.0,0.03,2.27,8.62,X40,BTT,IT
42,ITC,452.00,-39.19,43.0,X-LC,2.55,195796.0,-4342.0,25238.0,-0.60,-2.17,12.89,10.44,4.0,-0.17,1.40,3.27,X40,NTT,FMCG
83,VBL,671.64,-21.40,39.0,X-LC,10.46,283081.0,-32761.0,145419.0,0.01,-10.37,51.37,35.67,5.0,-0.23,2.02,2.04,X40N,ATH,FMCG
1,ABB,7934.00,-40.94,42.0,H-LC,7.43,246672.0,-14947.0,134165.0,-0.91,-5.71,54.39,45.57,7.0,-0.11,1.76,4.96,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-43.66,50.0,L-SC,29.06,81984.0,-31565.0,71613.0,-3.20,-27.80,87.35,35.27,268.0,-0.44,0.58,100.66,XR,NTT,HOTELS
49,MASFIN,398.61,-20.51,41.0,H-SC,10.01,90090.0,-7890.0,29495.0,-1.48,-8.05,32.74,22.05,152.0,-0.27,0.64,30.95,XR,ATH,FINANCE
50,NATIONALUM,244.55,-40.35,65.0,H-MC,14.55,119239.0,18735.0,10374.0,0.30,18.64,8.70,28.96,79.0,1.81,0.85,58.94,MH,ATH,METALS
78,UJJIVANSFB,60.00,108.87,64.0,H-SC,13.79,131706.0,-10773.0,30292.0,1.18,-7.56,23.00,13.70,163.0,-0.36,0.94,55.45,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,65.0,M-SC,16.39,145782.0,-3438.0,147619.0,-0.15,-2.30,101.26,96.62,208.0,-0.02,1.04,57.77,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-184.09,65.0,M-SC,16.39,145782.0,-3438.0,147619.0,-0.15,-2.30,101.26,96.62,208.0,-0.02,1.04,57.77,XY25,NTT,FINANCE
69,SURYODAY,240.00,64.47,62.0,H-SC,9.90,151286.0,-27785.0,93510.0,-1.78,-15.52,61.81,36.71,135.0,-0.30,1.08,50.11,XR,NTT,BANKS
32,HINDZINC,730.22,32.99,65.0,M-LC,8.24,217174.0,12098.0,100465.0,0.19,5.90,46.26,54.89,52.0,0.12,1.55,31.42,X5K,ATH,METALS
64,SHALBY,327.00,1175.92,55.0,M-SC,22.33,173631.0,-7512.0,52975.0,-0.60,-4.15,30.51,25.10,235.0,-0.14,1.24,39.15,XY24,NTT,HEALTHCARE
68,STARHEALTH,761.00,16.12,65.0,H-SC,7.81,258835.0,-22288.0,155146.0,-0.62,-7.93,59.94,47.26,144.0,-0.14,1.85,38.68,XY24,NTT,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.17
1,25,43.51
2,50,75.22


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.96
LC    32.33
MC    23.73
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.01
X40      14.63
X40N     11.27
XR       10.93
XY25     10.44
AR        8.12
OX40N     7.57
X200      1.75
MH        1.64
X5K       1.55
SR        1.11
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.44
X-LC    22.94
X-MC    17.21
M-SC    12.56
H-LC     4.61
H-MC     4.60
X-SC     4.53
M-LC     3.77
M-MC     1.61
L-SC     1.43
L-LC     1.01
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.82,-6.03,40.97
IT,12.83,-16.92,80.24
FINANCE,10.45,-15.27,70.93
MISC,6.93,-18.22,83.72
BANKS,6.44,-10.51,68.65
PAINTS,5.51,-19.42,37.22
ELECTRICAL,5.28,-11.11,50.10
INSURANCE,3.80,-4.06,42.96
AC,3.54,2.39,13.95


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3230559.0,22
XR,1376807.0,14
AR,1191452.0,9
X40,788848.0,10
X40N,655912.0,9
OX40N,585255.0,10
XY25,474636.0,7
SR,266055.0,2
X5K,100465.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3434612.0,25
M-SC,1281820.0,15
X-MC,1124220.0,12
X-LC,1025290.0,13
X-SC,544935.0,6
H-MC,364969.0,4
H-LC,294505.0,3
L-SC,259240.0,3
M-LC,236025.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1199695.0      6
           AR         811657.0      5
           XR         809473.0      7
M-SC       XY24       688391.0      6
X-MC       XY24       590962.0      4
X-LC       X40        490313.0      5
X-SC       X40N       305299.0      4
H-SC       OX40N      274769.0      4
           SR         266055.0      2
X-MC       X40        221266.0      4
X-LC       X40N       214276.0      3
H-LC       AR         213798.0      2
X-LC       XY24       212065.0      2
H-MC       XY24       182966.0      1
X-MC       XY25       175655.0      2
L-SC       XR         170100.0      2
M-MC       XY24       166698.0      1
M-SC       AR         165997.0      2
X-SC       XY24       162367.0      1
M-SC       OX40N      148273.0      4
           XY25       147619.0      1
X-MC       X40N       136337.0      2
M-SC       XR         131540.0      2
X-LC       XY25       108636.0      3
M-LC       XR         108145.0      1
           X5K        100465.0      1
H-MC       XR          98556.0      1
L-SC       OX40N       89140.0      1
H-LC       X200        80707.0      1
X-SC       X40         77269.0      1
H-MC       OX40N       73073.0      1
H-SC       MH          72963.0      1
L-MC       XR          58993.0      1
L-LC       XY25        42726.0      1
M-LC       XY24        27415.0      1
H-MC       MH          10374.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
